<a href="https://colab.research.google.com/github/CanteroAlann/machine-learning-projects/blob/main/TrabajoPractico3Parte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade category_encoders

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from category_encoders import TargetEncoder
import xgboost as xgb
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
train_df=pd.read_parquet('/content/drive/MyDrive/tp3 orga/train')

In [ ]:
train_df['popular'] = train_df['popular'].transform(lambda x : int(x))

In [ ]:
test_df = pd.read_parquet('/content/drive/MyDrive/tp3 orga/test')

In [ ]:
test_df['popular'] = test_df['popular'].transform(lambda x : int(x))

In [ ]:
#dividimos el dataset usando la siguiente condicion 
mask_train = train_df['timedelta'] > 120
X_train = train_df[mask_train]
X_valid = train_df[~mask_train]
y_train = X_train[X_train.columns[-2]]
y_valid = X_valid[X_valid.columns[-2]]
X_train.drop(columns=['popular','shares','timedelta','url'],inplace=True)
X_valid.drop(columns=['popular','shares','timedelta','url'],inplace=True)

Imputacion de nulos 

In [ ]:
#calculo la mediana para los valores numericos
title_subjectivity_median = X_train['title_subjectivity'].median()
abs_title_median = X_train['abs_title_subjectivity'].median()
max_positive_polarity_median = X_train['max_positive_polarity'].median()
n_tokens_title_median=X_train['n_tokens_title'].median()
n_tokens_content_median=X_train['n_tokens_content'].median()
n_unique_tokens_median=X_train['n_unique_tokens'].median()
num_videos_media=X_train['num_videos'].median()
n_non_stop_words_median=X_train['n_non_stop_words'].median()
n_non_stop_unique_tokens_median =X_train['n_non_stop_unique_tokens'].median()
num_hrefs_median=X_train['num_hrefs'].median()
num_self_hrefs_median = X_train['num_self_hrefs'].median()
num_imgs_median=X_train['num_imgs'].median()
num_keywords_median=X_train['num_keywords'].median()
kw_min_min_median=X_train['kw_min_min'].median()
kw_max_min_median=X_train['kw_max_min'].median()
kw_avg_min_median=X_train['kw_avg_min'].median()
kw_min_max_median=X_train['kw_min_max'].median()
kw_max_max_median=X_train['kw_max_max'].median()
kw_min_avg_median=X_train['kw_min_avg'].median()
self_reference_avg_sharess_median=X_train['self_reference_avg_sharess'].median()
LDA_00_median=X_train['LDA_00'].median()
LDA_01_median=X_train['LDA_01'].median()
LDA_02_median=X_train['LDA_02'].median()
LDA_03_median=X_train['LDA_03'].median()
global_sentiment_polarity_median=X_train['global_sentiment_polarity'].median()
global_rate_positive_words_median=X_train['global_rate_positive_words'].median()
global_rate_negative_words_median=X_train['global_rate_negative_words'].median()
rate_positive_words_median=X_train['rate_positive_words'].median()
rate_negative_words_median=X_train['rate_negative_words'].median()
avg_positive_polarity_median=X_train['avg_positive_polarity'].median()
avg_negative_polarity_median=X_train['avg_negative_polarity'].median()
max_negative_polarity_median=X_train['max_negative_polarity'].median()
min_negative_polarity_median=X_train['min_negative_polarity'].median()
abs_title_sentiment_polarity_median=X_train['abs_title_sentiment_polarity'].median()

In [ ]:
# reemplazamos en train 
X_train['title_subjectivity'].fillna(value=title_subjectivity_median,inplace=True)
X_train['abs_title_subjectivity'].fillna(value=abs_title_median,inplace=True)
X_train['max_positive_polarity'].fillna(value=max_positive_polarity_median,inplace=True)
X_train['content'].fillna(value="",inplace=True)
X_train['n_tokens_title'].fillna(value=n_tokens_title_median,inplace=True)
X_train['n_tokens_content'].fillna(value=n_tokens_content_median,inplace=True)
X_train['n_unique_tokens'].fillna(value=n_unique_tokens_median,inplace=True)
X_train['num_videos'].fillna(value=num_videos_media,inplace=True)
X_train['n_non_stop_words'].fillna(value=n_non_stop_words_median,inplace=True)
X_train['n_non_stop_unique_tokens'].fillna(value=n_non_stop_unique_tokens_median,inplace=True)
X_train['num_hrefs'].fillna(value=num_hrefs_median,inplace=True)
X_train['num_self_hrefs'].fillna(value=num_self_hrefs_median,inplace=True)
X_train['num_imgs'].fillna(value=num_imgs_median,inplace=True)
X_train['num_keywords'].fillna(value=num_keywords_median,inplace=True)
X_train['kw_min_min'].fillna(value=kw_min_min_median,inplace=True)
X_train['kw_max_min'].fillna(value=kw_max_min_median,inplace=True)
X_train['kw_avg_min'].fillna(value=kw_avg_min_median,inplace=True)
X_train['kw_min_max'].fillna(value=kw_min_max_median,inplace=True)
X_train['kw_max_max'].fillna(value=kw_max_max_median,inplace=True)
X_train['kw_min_avg'].fillna(value=kw_min_avg_median,inplace=True)
X_train['self_reference_avg_sharess'].fillna(value=self_reference_avg_sharess_median,inplace=True)
X_train['LDA_00'].fillna(value=LDA_00_median,inplace=True)
X_train['LDA_01'].fillna(value=LDA_01_median,inplace=True)
X_train['LDA_02'].fillna(value=LDA_02_median,inplace=True)
X_train['LDA_03'].fillna(value=LDA_03_median,inplace=True)
X_train['global_sentiment_polarity'].fillna(value=global_sentiment_polarity_median,inplace=True)
X_train['global_rate_positive_words'].fillna(value=global_rate_positive_words_median,inplace=True)
X_train['global_rate_negative_words'].fillna(value=global_rate_negative_words_median,inplace=True)
X_train['rate_positive_words'].fillna(value=rate_positive_words_median,inplace=True)
X_train['rate_negative_words'].fillna(value=rate_negative_words_median,inplace=True)
X_train['avg_positive_polarity'].fillna(value=avg_positive_polarity_median,inplace=True)
X_train['avg_negative_polarity'].fillna(value=avg_negative_polarity_median,inplace=True)
X_train['max_negative_polarity'].fillna(value=max_negative_polarity_median,inplace=True)
X_train['min_negative_polarity'].fillna(value=min_negative_polarity_median,inplace=True)
X_train['abs_title_sentiment_polarity'].fillna(value=abs_title_sentiment_polarity_median,inplace=True)
#Reemplazamos en valid
X_valid['n_unique_tokens'].fillna(value=n_unique_tokens_median,inplace=True)
X_valid['LDA_01'].fillna(value=LDA_01_median,inplace=True)
X_valid['global_rate_negative_words'].fillna(value=global_rate_negative_words_median,inplace=True)
X_valid['max_positive_polarity'].fillna(value=max_positive_polarity_median,inplace=True)
X_valid['max_negative_polarity'].fillna(value=max_negative_polarity_median,inplace=True) 
#Reemplazamos en test 
test_df['LDA_00'].fillna(value=LDA_00_median,inplace=True)
test_df['LDA_01'].fillna(value=LDA_01_median,inplace=True)
test_df['LDA_03'].fillna(value=LDA_01_median,inplace=True)
test_df['global_rate_positive_words'].fillna(value=global_rate_positive_words_median,inplace=True)
test_df['rate_negative_words'].fillna(value=rate_negative_words_median,inplace=True)
test_df['content'].fillna(value="",inplace=True)

In [ ]:
# imputacion de nulos para variables nominales 
def nominal_nulls(column,columns_names,dataset):
  nulls_index = dataset.index[dataset[column].isnull()].to_list()
  for index in nulls_index:
    for name in columns_names:
      if dataset.loc[index,name]:
        dataset.at[index,column] = 0.0
      else:
        dataset.at[index,column] = 1.0

In [ ]:
channels_list = list(["data_channel_is_lifestyle","data_channel_is_entertainment","data_channel_is_bus","data_channel_is_socmed","data_channel_is_tech","data_channel_is_world"])
days_list = list(['weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday','weekday_is_friday','weekday_is_saturday','weekday_is_sunday'])

In [ ]:
nominal_nulls('data_channel_is_bus',channels_list,X_train)
nominal_nulls('data_channel_is_socmed',channels_list,X_train)
nominal_nulls('data_channel_is_tech',channels_list,X_train)
nominal_nulls('weekday_is_monday',days_list,X_train)
nominal_nulls('weekday_is_tuesday',days_list,X_train)
nominal_nulls('weekday_is_wednesday',days_list,X_train)
nominal_nulls('weekday_is_friday',days_list,X_train)
nominal_nulls('weekday_is_saturday',days_list,X_train)
nominal_nulls('weekday_is_thursday',days_list,test_df)

In [ ]:
#dividimos el set de test
X_test = test_df.drop(columns=['popular','shares','timedelta','url'])
y_test = test_df[test_df.columns[-2]]

In [ ]:
ohe_surprise1 = OneHotEncoder(drop='first')
encoded_surprise1 = ohe_surprise1.fit_transform(X_train[['surprise1']]).todense().astype(int)
encoded_surprise1_valid = ohe_surprise1.transform(X_valid[['surprise1']]).todense().astype(int)
encoded_surprise1_test = ohe_surprise1.transform(X_test[['surprise1']]).todense().astype(int)

In [ ]:
count_vectorizer = CountVectorizer(lowercase=True,max_features=100,stop_words='english')
encoded_content = count_vectorizer.fit_transform(X_train['content']).toarray()
encoded_content_valid = count_vectorizer.transform(X_valid['content']).toarray()
encoded_content_test = count_vectorizer.transform(X_test['content']).toarray()

In [ ]:
X_train.content=encoded_content
X_valid.content=encoded_content_valid
X_test.content=encoded_content_test

In [ ]:
X_train.surprise1 = encoded_surprise1
X_valid.surprise1 = encoded_surprise1_valid
X_test.surprise1 = encoded_surprise1_test

In [ ]:
bin_enc_surprise2 = LabelBinarizer()
encoded_surprise2 = bin_enc_surprise2.fit_transform(X_train['surprise2'])
encoded_valid_surprise2 = bin_enc_surprise2.transform(X_valid['surprise2'])
encoded_test_surprise2 = bin_enc_surprise2.transform(X_test['surprise2'])

In [ ]:
X_train.surprise2 = encoded_surprise2
X_valid.surprise2 = encoded_valid_surprise2
X_test.surprise2 = encoded_test_surprise2

Usamos el mismo modelo buscando los hiper parametros

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'learning_rate': (0.2,0.3,0.4),
    'max_depth': [6,10,20,30,50],
    'min_split_loss' : (0,2,4),
    'colsample_bytree': (0.2,0.4,1),
    'n_estimators': [5,20,30,40,100,150]
}
# Create a based model
xgb = xgb.XGBClassifier(random_state=20)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb, param_grid = param_grid, 
                          cv = 2, n_jobs = -1,scoring='roc_auc')

In [ ]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=2, estimator=XGBClassifier(random_state=20), n_jobs=-1,
             param_grid={'colsample_bytree': (0.2, 0.4, 1),
                         'learning_rate': (0.2, 0.3, 0.4),
                         'max_depth': [6, 10, 20, 30, 50],
                         'min_split_loss': (0, 2, 4),
                         'n_estimators': [5, 20, 30, 40, 100, 150]},
             scoring='roc_auc')

In [ ]:
preds = grid_search.predict(X_valid)
mean_absolute_error(y_valid, preds)

0.16964285714285715

In [ ]:
proba_preds = grid_search.predict_proba(X_valid)[:,1]
roc_auc_score(y_valid,proba_preds)

0.726993977428682